In [ ]:
# !pip install nltk
# import nltk
# nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
import pandas as pd
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords


# Data

In [ ]:
data = pd.read_csv("data/ACL_data.csv").drop(columns = 'Unnamed: 0' )
data.head()

In [ ]:
data.info()

In [ ]:
data.ABSTRACT.tolist()[:3]

## Cleaning and tokenize

In [99]:
# En este caso particular no hace falta limpiar los textos
def clean_text(text):
    if isinstance(text, pd.Series):
        return text.str.strip().values
    elif isinstance(text, str):
        return text.strip()

def tokenizer(text):
    #return [w for w in word_tokenize(text) if w.isalpha()] # si solo nos interesan palabras
    if isinstance(text, str):
        return word_tokenize(text)
    else:
        return text.apply(word_tokenize)



In [108]:
# word_tokenize(clean_text(data.ABSTRACT))
tokenizer(clean_text(data.ABSTRACT))


AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [ ]:
data.ABSTRACT.apply(word_tokenize)

In [ ]:
x = data.ABSTRACT.apply(word_tokenize)

In [ ]:
tokenizer(clean_text(data.ABSTRACT))#[:10]

La funcion anterior rompe con series y dataframes

In [92]:
print( type( clean_text( data.ABSTRACT[0]) ))
print( type(clean_text( data.ABSTRACT[:1]) ) )

# tokenizer(clean_text(data.ABSTRACT[:1] ))#[:10]

<class 'str'>
<class 'numpy.ndarray'>


In [ ]:
data.ABSTRACT[:1].values
# data.iloc[:1]

## Stoplist

In [ ]:
stoplist = stopwords.words("english")
stoplist[:15]

In [ ]:
stoplist_tokenized = []
for w in stoplist:
    stoplist_tokenized = stoplist_tokenized + tokenizer(w)
stoplist_tokenized[:15]